<b><font size=20, color='#A020F0'>Xarray

Hannah Zanowski<br>
10/11/21<br>

#### Resources
[Xarray Documentation](http://xarray.pydata.org/en/stable/)<br>
[Xarray API reference](http://xarray.pydata.org/en/stable/api.html)<br>

# A little about Xarray

Xarray is an extra-glorious library for working with multi-dimensional data that was inspired by Pandas. It uses numpy's N-D arrays, but with _labels_, which makes working with multi-dimensional data much more inuitive and streamlined. It is excellent at handling gridded data, including the [netCDF](https://www.unidata.ucar.edu/software/netcdf/) file format, which is a very common format for atmospheric and oceanic science data. Xarray is one of the packages I use the most in my work.

Let's begin by importing xarray (and a few others):

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

---

## 1. DataArrays
[DataArray](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.html) objects are a core data structure in xarray. There are 4 key elements associated with a DataArray:
1. `values`: the numpy array that holds the data
2. `dims`: the names for each axis of your data (e.g., 'x','y')
3. `coords`: the coordinates that label each data point (e.g., arrays for latitude, dates)
4. `attrs`: the attributes of the data (e.g., metadata like units, descriptions of the variables, etc)

### Creating an xarray DataArray
It's straightforward to make a DataArray. Here's what one looks like in its simplest form:

In [ ]:
da=xr.DataArray(np.arange(2,22,2))
da

But this isn't particularly instructive. We need to add arguments for the dimensions, coordinates, and some attributes in order for the values in our DataArray to be meaningful. Let's use our Argo float data from Homework1 to make a DataArray:

In [ ]:
argo=np.load('../Homework/20210901_IndianOcean.npz')
list(argo.keys())

In [ ]:
#Make the argo temperature data a data array
da=xr.DataArray(argo['temperature'],dims=['pressure'],coords={'pressure':argo['pressure']})
da

Let's add a units attribute to our DataArray:

You can access the values, coordinates, dimensions,and attributes of your DataArray with the following commands:

Like Pandas, there is built-in plotting with xarray:

In [ ]:
da.plot(y='pressure',yincrease=False)

---

## 2. Datasets
[Dataset](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.html) objects are another core data structure in xarray. Like the Pandas Dataframe, an xarray Dataset is just a container for a collection of xarray DataArrays that share coordinates.

### Creating an xarray Dataset
Let's use our argo data to make our first xarray Dataset. Datasets need the following information:
1. `data_vars`: a dictionary `{}` that maps variable names to the data they represent
2. `coords`: the coordinates of the data variables<br>

As with DataArrays, you can also set attributes with `attrs`. Attributes are techincally not _required_ to create a Dataset (just like for DataArrays), but you should include them! They can be set globally for the dataset and for individual data variables. For some common attributes, check out the [CF Conventions](http://cfconventions.org/Data/cf-conventions/cf-conventions-1.7/cf-conventions.html#_description_of_the_data)

<b>Note:</b> The data that you provide in `data_vars` can be DataArrays, pandas objects, etc, or you can create new data as long as it is a tuple of the form `(dims, data[, attrs])`, which xarray will interpret.

In [ ]:
ds=xr.Dataset(data_vars={'temp':('pressure',argo['temperature'],{'units':'degC'}),
                         'salt':('pressure',argo['salinity'],{'units':'PSU'})},
             coords={'pressure':('pressure',argo['pressure'])},
             attrs={'Description':'Argo salinity and temperature data from the equatorial Indian Ocean'})
ds

Awesome! But what did we just _do_? Let's unpack the `data_vars` argument in `xr.Dataset()`:<br>

As mentioned above, `data_vars` is a dictionary, and it maps the names of variables to the data from those variables. It does this in the form {key:tuple}. For just the variable 'temp' this looks like `{'temp':('pressure',argo['temperature'],{'units':'degC'})}`<br>

Here `'temp'` is the dictionary key (the name of the variable) and the tuple contains the pieces of information we are assigning to that variable name. In this case the tuple for `'temp'` contains three pieces of information:
1. The dimensions--in this case the only dimension our data has is `'pressure'`<b><font color='red'>NOTE:</font></b> If your data variable has more than 1 dimension, you will need to provide these as a tuple, e.g. (dim1,dim2,...)
2. The data itself--in this case it's the temperature data from our argo float, `argo['temperature']`
3. The attributes of our data variable, which is _another_ dictionary--in this case we only gave the argo temeprature data variable a 'units' attribute. Because this is another dictionary, it maps the key (`'units'`) to the value (`'degC'`)

#### Adding/dropping data variables
You can add data variables to a dataset after you've created it:

In [ ]:
#Add a data variable for the square of the temperature
ds['temp_squared']=ds.temp*ds.temp
ds

You can also drop data variables:

In [ ]:
ds=ds.drop('temp_squared')
ds

#### Using to_dataset() with DataArrays
If you have some defined DataArrays, you can use xarray's [to_dataset()](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.to_dataset.html) to create a dataset directly from them:

In [ ]:
ds2=da.to_dataset(name='temp')
ds2

---

## 3. Working with xarray datasets
To show you the real power of xarray, we'll be using [HadISST1](https://www.metoffice.gov.uk/hadobs/hadisst/data/download.html) sea surface temperature data from Jan 2000 to Dec 2020 in the following examples

### Reading in data
You can read in [netCDF](https://www.unidata.ucar.edu/software/netcdf/) data using [open_dataset](http://xarray.pydata.org/en/stable/generated/xarray.open_dataset.html) (or [open_mfdataset](http://xarray.pydata.org/en/stable/generated/xarray.open_mfdataset.html) to open multiple files at once!)

### Accessing data

There's a lot of information in this dataset, but how do we interpret it? Let's start by getting the attributes of the data variable `sst`:

Because the attributes are a dictionary, we can also access a specific attribute if we know its name:

In [ ]:
#Get only the units attribute of SST


Acessing specific data variables is much like in pandas when we access columns:

In [ ]:

#or
#ds.sst

#### Selecting data
You can [select data from a Dataset or DataArray using](http://xarray.pydata.org/en/stable/user-guide/indexing.html) `.sel()` or `.isel()`. Like pandas, `.sel()` allows you to select based on variables names, etc and `.isel()` allows you to select data based on its integer index.

In [ ]:

#or
#ds['sst'].sel({'time':'2015'})

In [ ]:
#Get SST data at the first time

#or
#ds['sst'].isel({'time':0})

Both `.sel()` and `.isel()` can take multiple inputs:

In [ ]:
#Choose sst data at a specific time and point


But what if you don't remember or know the _exact_ coordinate values? You can set the 'method' keyword argument!

#### Slicing
If you want to select a range of data you need to use xarray's `slice()`. Let's use slice to select a subset of data for our entire dataset.

In [ ]:
ds_slice=ds.sel(time=slice('2015-01-16','2016-12-16'),longitude=slice(30.5,179.5),latitude=slice(45,None))
#or
#ds_slice=ds.sel({'time':slice('2015-01-16','2016-12-16'),'longitude':slice(30.5,60.5),'latitude':slice(45,None)})

Let's make a quick 2D plot of SST using the first date:

In [ ]:


#This is equivalent to
#ds_slice.sst.isel(time=0).plot()
#ds.slice.sst.sel(time='2015-01-16').plot()

Hmmmm, this isn't quite what we wanted...If you look closely at the dataset output, you'll see that this is because of the order that our latitude values are provided in the dataset--they go from 90N to 90S, _not_ 90S to 90N! We can fix this a couple of different ways:

1. We can change the slice argument for latitude slightly: slice(None,45)
2. We can just reset the latitude coordinate for the entire dataset

Let's use option 2 so we don't have to deal with it again. Here we'll be using [reindex()](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.reindex.html) to reindex the dataset based on our new latitude coordinate that starts at -89.5 and ends at 89.5:

In [ ]:
ds=ds.reindex(latitude=list(reversed(ds.latitude)))
ds

#### Filtering with where()
The other, more obvious thing you probably noticed about your plot is that the range of SST values is enormous and unphysical. This is because this dataset has some missing values under sea ice, and these are set to -1000. Let's get rid of those using xarray's [where()](http://xarray.pydata.org/en/stable/generated/xarray.where.html) function. `where()` takes as input a condition on your data and replaces the values that <font color='red'><b>_DO NOT_</b></font> meet that condition with nan. If you want those values to be replaced with a value other than nan, use the `other` keyword argument.

Before we use `where()` to get rid of the missing values, we want to make a mask for land, which is currently set as nan. If we don't keep track of the land separately, then once we use `where()` both the missing values and land values will be the same.

In [ ]:
land=np.isnan(ds.sst[0])

In [ ]:
#make a quick plot with matplotlib's pcolormesh


Now that we have our land mask let's use `.where()` to turn the missing values into nan:

In [ ]:
ds=ds.where(ds.sst>-1000)

Let's add our land mask to the dataset:

In [ ]:
ds['land']=land
ds

And finally, let's make another plot, but now using our SSTs without the missing values:

In [ ]:
fig,ax=plt.subplots()
#add land first
ax.pcolormesh(ds.land,cmap=plt.cm.gray_r)
#then plot over the land with SSTs
cs=ax.pcolormesh(ds['sst'].sel(time='2015-01-16').squeeze(),cmap=plt.cm.plasma)
cbar=plt.colorbar(cs)

<b>Note:</b> In this particular case, if all you wanted to do was plot the data and _not_ bother with `where()` you could just do the following once you read in your data:

```bash
cmap=plt.cm.plasma.copy()
cmap.set_under('0.5') #set values under the minimum value to be gray
fig,ax=plt.subplots()
#plot the SSTs, but set the min and max values for the colorbar
cs=ax.pcolormesh(ds['sst'].sel(time='2015-01-16').squeeze(),cmap=cmap,vmin=-2, vmax=ds.sst.max())
cbar=plt.colorbar(cs)
```

#### Using where() for multiple conditions
If you want to filter your data based on multiple conditions at once, all you need to do is chain several `where()` functions together:

In [ ]:
#Get only the SST values that fall between 5˚C and 20˚C that are south of the Equator


In [ ]:
#Plot your SST subset at a single time


### Doing computations on xarray Datasets and DataArrays

When you do computations on xarray objects, the coordinates are preserved. You can apply most numpy functions to Datasets and Datarrays. Like Pandas though, not every function from a package is designed to work with xarray objects. You can still apply these functions to your xarray data, but the output will not be an xarray DataArray or Dataset unless you use [apply_ufunc](http://xarray.pydata.org/en/stable/generated/xarray.apply_ufunc.html)

In [ ]:
sst_mn=ds.sst.mean(dim='time') #you can supply dimension names instead of numerical axes indices

In [ ]:
sst_mn.plot()
plt.gca().set_facecolor('0.7') #make the plot background light gray

If you supply more than one dimension at a time, it needs to be a tuple:

In [ ]:
#Plot the zonal mean, time mean SSTs


In [ ]:
#Plot SST anomalies at a point in the equatorial Pacific Ocean
sst_pac=ds.sst.sel(latitude=0.5,longitude=-150.5)
sst_anom=sst_pac-sst_pac.mean()
sst_anom.plot()

#### Broadcasting no more!
Because our data is labeled, xarray can handle broadcasting internally. The hell that is broadcasting with numpy arrays is not a problem in xarray!

In [ ]:
print(ds.longitude.shape)
print(ds.sst.shape)

In [ ]:
sst_lon_weighted=ds.sst*np.cos(ds.longitude*np.pi/180)
sst_lon_weighted.shape

In [ ]:
#plot the longitude-weighted SSTs at one time


#### Groupby
[Groupby](http://xarray.pydata.org/en/stable/user-guide/groupby.html) also exists in xarray, and it works much the same way it does in pandas:

In [ ]:
#Get the mean SST for each year
sst_ann_mn=ds.sst.groupby('time.year').mean('time')
sst_ann_mn.shape

In [ ]:
#Get seasonal SSTs
sst_seasonal_mn=ds.sst.groupby('time.season').mean('time')

In [ ]:
#Look at the dimensions of seasonal mean SSTs


In [ ]:
#Access the contents of the 'season' dimension


In [ ]:
#Plot the difference between (Southern Hemisphere) summer and winter SSTs
plt.gca().set_facecolor('0.7')
cs=plt.contourf(ds.longitude, ds.latitude,sst_seasonal_mn.sel(season='DJF')-sst_seasonal_mn.sel(season='JJA'),
                  cmap=plt.cm.RdBu_r,levels=np.arange(-10,11,1),extend='both')
plt.colorbar()
plt.title('SH Summer (DJF) - Winter (JJA) SSTs')

### Saving your data
You can [save your xarray Datasets in many formats](http://xarray.pydata.org/en/stable/user-guide/io.html), including netcdf. To save your Dataset to a netcdf file, use [to_netcdf](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.to_netcdf.html). For practice, let's save one year of SSTs to a netcdf file:

In [ ]:
ds_to_save=ds.sel(time=slice('2015-01','2015-12'))

Now try opening your saved dataset:

---